### Install Data Processing Package and BeautifulSoup

In [4]:
from bs4 import BeautifulSoup
import lxml
import html5lib
import requests
import pandas as pd
from requests import get

In [5]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

### Use BeautifulSoup to extract text message

In [6]:
soup = BeautifulSoup(source,'lxml')
#print(soup.prettify())

In [7]:
body = soup.find('body')
#print(body.prettify())

In [8]:
body_content = body.find('div',class_='mw-parser-output').table.tbody
print(len(body_content))
data_list = []

580


### Find Table data that includes PostalCode/Borough/Neighborhood

In [9]:
for idx, tr in enumerate(body_content.find_all('tr')):
    if idx != 0:
        tds = tr.find_all('td')
        tds0 = tds[0]
        tds1 = tds[1].find('a')
        tds2 = tds[2].find('a')
        if tds1 == None and tds2 == None:   
            data_list.append({
            'PostalCode': tds0.contents[0],
            'Borough': tds[1].contents[0],
            'Neighborhood':tds[2].contents[0]
            })
        elif tds1 != None and tds2 == None:
            data_list.append({
            'PostalCode': tds0.contents[0],
            'Borough': tds1.text,
            'Neighborhood':tds[2].contents[0]
            })
        elif tds1 == None and tds2 != None:
            data_list.append({
            'PostalCode': tds0.contents[0],
            'Borough': tds[1].contents[0],
            'Neighborhood':tds2.text
            })
        else:
             data_list.append({
             'PostalCode': tds0.contents[0],
              'Borough': tds1.text,
              'Neighborhood':tds2.text
              })

In [10]:
df = pd.DataFrame(data_list,columns = ['PostalCode','Borough','Neighborhood'])

### Clean data: Clearing Borough data that has 'not assigned', Deleting Spaces, Joining the Neighborhoods that have the same Postal Code

In [11]:
df = df[(True^df['Borough'].isin(['Not assigned']))]

In [12]:
for x in df['Neighborhood']:
    if x in ('Not assigned','Not assigned\n'):
        df.loc[df.Neighborhood == x,'Neighborhood']=df.loc[df.Neighborhood == x,'Borough']
    else:
        pass

In [13]:
df['Neighborhood']=df['Neighborhood'].str.replace('\n','')
#print(df.to_string()) #a way to display all rows of data

In [14]:
dataset = df.groupby('PostalCode')['Neighborhood'].apply(','.join).reset_index()
dataset = dataset.join(df.set_index('PostalCode')['Borough'], on='PostalCode').drop_duplicates()  ##set PostalCode column as Index!
Torontodata = pd.DataFrame(dataset,columns = ['PostalCode','Borough','Neighborhood'])
Torontodata = Torontodata.sample(frac=1).reset_index(drop=True)

In [15]:
Torontodata.shape

(103, 3)